# 📚 M3c‴: Optimized HDC Semantic Transfer — Student

## Key insights from M2.5e applied:

| M2.5e Finding | Implementation |
|---------------|----------------|
| **Sharp curriculum: easy → hard** | Train on easy first, then add hard |
| **Constant LR (NO decay!)** | Fixed learning rate throughout |
| **Distance from centroid = difficulty** | Use difficulty scores from Teacher |
| **+8.1% improvement potential** | Target: 59% → 70%+ transfer efficiency |

**GPU:** L4 recommended

---

In [ ]:
!pip install -q firebase-admin
!pip install -q sentence-transformers
print("✅ Dependencies installed")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
import numpy as np
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import base64
import matplotlib.pyplot as plt

import firebase_admin
from firebase_admin import credentials, db

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n📚 M3c‴ Student — Sharp Curriculum + Constant LR")

In [ ]:
# ============================================================
# FIREBASE CREDENTIALS - INSERT YOUR KEY HERE
# ============================================================
FIREBASE_CREDENTIALS = "INSERT KEY HERE"

FIREBASE_DATABASE_URL = "https://resonance-m3-default-rtdb.europe-west1.firebasedatabase.app"

if not firebase_admin._apps:
    cred = credentials.Certificate(FIREBASE_CREDENTIALS)
    firebase_admin.initialize_app(cred, {'databaseURL': FIREBASE_DATABASE_URL})
print("✅ Firebase initialized")

## Download Knowledge Packet

In [ ]:
print("📡 Checking for Teacher...")

ref = db.reference('sep_m3c_hdc_v3/status')
status = ref.get()

if status and status.get('teacher_ready', False):
    print(f"✅ Teacher ready!")
    print(f"   HDC dim: {status.get('hdc_dim')}")
    print(f"   Examples: {status.get('num_examples')}")
    print(f"   Curriculum: {status.get('curriculum')}")
    print(f"   Teacher accuracy: {status.get('teacher_accuracy', 0):.1%}")
else:
    print("⚠️ Teacher not ready!")
    raise Exception("Run Teacher first")

In [ ]:
print("\n📥 Downloading knowledge packet...")

ref = db.reference('sep_m3c_hdc_v3/knowledge_packet')
knowledge_packet = ref.get()

HDC_DIM = knowledge_packet['hdc_dim']
projection_weights = knowledge_packet['projection_weights']
teacher_acc_before = knowledge_packet['teacher_accuracy_before']
teacher_acc_after = knowledge_packet['teacher_accuracy_after']

print(f"\n📋 Knowledge Packet:")
print(f"   Version: {knowledge_packet.get('version')}")
print(f"   HDC dim: {HDC_DIM}")
print(f"   Curation: {knowledge_packet.get('curation_method')}")
print(f"   Curriculum: {knowledge_packet.get('curriculum_strategy')}")
print(f"   Easy: {knowledge_packet.get('num_easy')}, Hard: {knowledge_packet.get('num_hard')}")
print(f"   Teacher: {teacher_acc_before:.1%} → {teacher_acc_after:.1%}")

In [ ]:
def decompress_hdc_packet(compressed_examples, hdc_dim):
    examples = []
    for ex in tqdm(compressed_examples, desc="Decompressing"):
        packed = np.frombuffer(base64.b64decode(ex['hdc_packed']), dtype=np.uint8)
        unpacked = np.zeros(hdc_dim, dtype=np.int8)
        for j in range(4):
            unpacked[j::4] = (packed >> (j * 2)) & 0b11
        hdc_vec = unpacked.astype(np.int8) - 1
        examples.append({
            'hdc_vector': hdc_vec,
            'label': ex['label'],
            'difficulty': ex['difficulty'],
            'curriculum': ex['curriculum']
        })
    return examples

hdc_examples = decompress_hdc_packet(knowledge_packet['examples'], HDC_DIM)

# Separate easy and hard
easy_examples = [ex for ex in hdc_examples if ex['curriculum'] == 'easy']
hard_examples = [ex for ex in hdc_examples if ex['curriculum'] == 'hard']

print(f"\n✅ Decompressed: {len(easy_examples)} easy + {len(hard_examples)} hard")

## HDC Encoder (with Teacher's projection)

In [ ]:
class LearnedHDCEncoder(nn.Module):
    def __init__(self, hdc_dim=10000, semantic_dim=384):
        super().__init__()
        self.hdc_dim = hdc_dim
        self.semantic_encoder = SentenceTransformer('all-MiniLM-L6-v2')
        self.projection = nn.Sequential(
            nn.Linear(semantic_dim, hdc_dim),
            nn.Tanh(),
        )
    
    def load_projection_weights(self, weights_dict):
        self.projection[0].weight.data = torch.tensor(weights_dict['weight'], dtype=torch.float32)
        self.projection[0].bias.data = torch.tensor(weights_dict['bias'], dtype=torch.float32)
        print("✅ Loaded Teacher's projection")
    
    def encode(self, texts, quantize=True):
        if isinstance(texts, str):
            texts = [texts]
        with torch.no_grad():
            semantic = self.semantic_encoder.encode(texts, convert_to_tensor=True, show_progress_bar=False)
            if semantic.device != next(self.projection.parameters()).device:
                semantic = semantic.to(next(self.projection.parameters()).device)
            hdc_vecs = self.projection(semantic)
        
        if quantize:
            hdc_np = hdc_vecs.cpu().numpy()
            threshold = 0.33
            return np.where(hdc_np > threshold, 1, np.where(hdc_np < -threshold, -1, 0)).astype(np.int8)
        return hdc_vecs.cpu().numpy()

hdc_encoder = LearnedHDCEncoder(hdc_dim=HDC_DIM).to(device)
hdc_encoder.load_projection_weights(projection_weights)
hdc_encoder.eval()

## Improved Student Model

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.1):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
        )
        self.activation = nn.GELU()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        return self.dropout(self.activation(x + self.block(x)))


class ImprovedHDCClassifier(nn.Module):
    def __init__(self, hdc_dim, hidden_dim=1024, num_classes=2, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Sequential(
            nn.Linear(hdc_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
        )
        self.res_blocks = nn.Sequential(
            ResidualBlock(hidden_dim, dropout),
            ResidualBlock(hidden_dim, dropout),
        )
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        self.num_params = sum(p.numel() for p in self.parameters())
    
    def forward(self, x):
        x = self.input_proj(x)
        x = self.res_blocks(x)
        return self.classifier(x)

student = ImprovedHDCClassifier(hdc_dim=HDC_DIM, hidden_dim=1024, dropout=0.1).to(device)
print(f"\n📚 Student: {student.num_params:,} parameters")

## Prepare Test Data

In [ ]:
print("Loading test data...")
dataset = load_dataset("glue", "sst2")

TEST_SIZE = 500
test_texts = dataset['validation']['sentence'][:TEST_SIZE]
test_labels = dataset['validation']['label'][:TEST_SIZE]

# Encode to HDC
print("\n🔄 Encoding test texts...")
test_hdc = []
for i in tqdm(range(0, len(test_texts), 64), desc="Encoding"):
    batch = list(test_texts[i:i+64])
    vecs = hdc_encoder.encode(batch, quantize=True)
    test_hdc.extend(vecs)
test_hdc = np.array(test_hdc)

print(f"✅ Test: {test_hdc.shape}")

In [ ]:
class HDCDataset(Dataset):
    def __init__(self, hdc_vectors, labels):
        self.hdc = torch.tensor(hdc_vectors, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.hdc[idx], self.labels[idx]

test_dataset = HDCDataset(test_hdc, test_labels)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for hdc, labels in dataloader:
            hdc = hdc.to(device)
            outputs = model(hdc)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    return accuracy_score(all_labels, all_preds)

print("📊 Student BEFORE training...")
acc_before = evaluate(student, test_loader)
print(f"✅ Accuracy BEFORE: {acc_before:.1%}")

## Sharp Curriculum Training (M2.5e)

**Key insights applied:**
1. **Sharp curriculum:** Easy examples first, then hard
2. **Constant LR:** NO learning rate decay!
3. **Phase transitions:** Clear boundaries between easy/hard

In [ ]:
print("\n" + "="*60)
print("🧠 SHARP CURRICULUM TRAINING (M2.5e method)")
print("="*60)
print("\n📚 Strategy:")
print("   Phase 1 (epochs 1-15):  EASY examples only")
print("   Phase 2 (epochs 16-35): EASY + HARD examples")
print("   Phase 3 (epochs 36-50): All examples")
print("\n⚠️  CONSTANT LR = 5e-4 (no decay!)")

In [ ]:
# Prepare curriculum datasets
easy_hdc = np.array([ex['hdc_vector'] for ex in easy_examples])
easy_labels = np.array([ex['label'] for ex in easy_examples])
hard_hdc = np.array([ex['hdc_vector'] for ex in hard_examples])
hard_labels = np.array([ex['label'] for ex in hard_examples])

# Combined dataset
all_hdc = np.vstack([easy_hdc, hard_hdc])
all_labels = np.concatenate([easy_labels, hard_labels])

print(f"\n📊 Curriculum data:")
print(f"   Easy: {len(easy_hdc)}")
print(f"   Hard: {len(hard_hdc)}")
print(f"   Total: {len(all_hdc)}")

In [ ]:
# Training configuration
NUM_EPOCHS = 50
PHASE1_END = 15   # Easy only
PHASE2_END = 35   # Easy + Hard
# Phase 3: All (same as phase 2 in our case)

# ⚠️ CONSTANT LR - Key insight from M2.5e!
LEARNING_RATE = 5e-4

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(student.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
# NO scheduler! Constant LR preserves adaptability (M2.5e)

history = {'train_loss': [], 'train_acc': [], 'test_acc': [], 'phase': []}
best_test_acc = 0
best_state = None

In [ ]:
for epoch in range(NUM_EPOCHS):
    # Determine which data to use based on curriculum phase
    if epoch < PHASE1_END:
        # Phase 1: Easy only
        phase = 1
        train_data = HDCDataset(easy_hdc, easy_labels)
    else:
        # Phase 2 & 3: Easy + Hard
        phase = 2 if epoch < PHASE2_END else 3
        train_data = HDCDataset(all_hdc, all_labels)
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)
    
    # Training
    student.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for hdc, labels in train_loader:
        hdc = hdc.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = student(hdc)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(student.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    train_loss = total_loss / len(train_loader)
    train_acc = correct / total
    test_acc = evaluate(student, test_loader)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)
    history['phase'].append(phase)
    
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_state = student.state_dict().copy()
    
    # Log phase transitions and regular progress
    if epoch == 0 or epoch == PHASE1_END or epoch == PHASE2_END or (epoch + 1) % 10 == 0:
        phase_str = f"Phase {phase}"
        if epoch == PHASE1_END:
            phase_str = "→ Phase 2 (adding HARD)"
        elif epoch == PHASE2_END:
            phase_str = "→ Phase 3 (all data)"
        
        data_size = len(easy_hdc) if phase == 1 else len(all_hdc)
        print(f"Epoch {epoch+1:2d}/{NUM_EPOCHS} | {phase_str:25} | "
              f"Data: {data_size:4d} | "
              f"Loss: {train_loss:.4f} | "
              f"Test: {test_acc:.1%} | "
              f"Best: {best_test_acc:.1%}")

# Load best model
student.load_state_dict(best_state)
print(f"\n✅ Training complete! Best: {best_test_acc:.1%}")

In [ ]:
# Final evaluation
acc_after = evaluate(student, test_loader)
print(f"\n📊 Final accuracy: {acc_after:.1%}")

## Results Analysis

In [ ]:
student_improvement = acc_after - acc_before
teacher_improvement = teacher_acc_after - teacher_acc_before
transfer_efficiency = (student_improvement / teacher_improvement) * 100 if teacher_improvement > 0 else 0

print("\n" + "="*60)
print("📊 RESULTS: CURRICULUM-OPTIMIZED HDC TRANSFER")
print("="*60)

print(f"\n🎓 TEACHER:")
print(f"   {teacher_acc_before:.1%} → {teacher_acc_after:.1%} (+{teacher_improvement:.1%})")

print(f"\n📚 STUDENT (HDC only, curriculum trained):")
print(f"   {acc_before:.1%} → {acc_after:.1%} (+{student_improvement:.1%})")

print(f"\n🔄 TRANSFER EFFICIENCY: {transfer_efficiency:.1f}%")

print(f"\n📈 COMPARISON:")
print(f"   M3c″ (random, no curriculum):     59.2%")
print(f"   M3c‴ (curriculum + constant LR): {transfer_efficiency:.1f}%")
print(f"   Improvement: +{transfer_efficiency - 59.2:.1f}%")

In [ ]:
# Verdict
if transfer_efficiency >= 80:
    verdict = "EXCELLENT"
    emoji = "🎉"
elif transfer_efficiency >= 70:
    verdict = "VERY GOOD"
    emoji = "🌟"
elif transfer_efficiency >= 65:
    verdict = "GOOD"
    emoji = "✅"
else:
    verdict = "IMPROVED"
    emoji = "📈"

print(f"\n{emoji} VERDICT: {verdict}")
print(f"\n   Curriculum learning applied: ✅")
print(f"   Constant LR (no decay): ✅")
print(f"   HDC K-means curation: ✅")
print(f"   Text seen by Student: ZERO")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Learning curve with phase markers
ax = axes[0]
ax.plot(history['test_acc'], label='Test Accuracy', color='orange', linewidth=2)
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Random')
ax.axvline(x=PHASE1_END, color='red', linestyle=':', alpha=0.7, label='Phase 2 (add hard)')
ax.axvline(x=PHASE2_END, color='purple', linestyle=':', alpha=0.7, label='Phase 3')
ax.fill_between(range(PHASE1_END), 0, 1, alpha=0.1, color='green', label='Easy only')
ax.fill_between(range(PHASE1_END, NUM_EPOCHS), 0, 1, alpha=0.1, color='red')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Sharp Curriculum Learning')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_ylim(0.4, 1.0)

# Comparison bars
ax = axes[1]
versions = ['M3c″\n(baseline)', 'M3c‴\n(curriculum)']
efficiencies = [59.2, transfer_efficiency]
colors = ['lightcoral', 'lightgreen']
bars = ax.bar(versions, efficiencies, color=colors, edgecolor='gray')
ax.set_ylabel('Transfer Efficiency (%)')
ax.set_title('Transfer Efficiency Comparison')
ax.set_ylim(0, 100)
ax.axhline(y=80, color='green', linestyle='--', alpha=0.5, label='Target: 80%')
for bar, val in zip(bars, efficiencies):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{val:.1f}%', ha='center', fontsize=12, fontweight='bold')
ax.legend()

# Accuracy comparison
ax = axes[2]
models = ['Teacher\n(text)', 'M3c″\n(HDC)', 'M3c‴\n(curriculum)']
accs = [teacher_acc_after, 0.734, acc_after]
colors = ['lightblue', 'lightyellow', 'lightgreen']
bars = ax.bar(models, accs, color=colors, edgecolor='gray')
ax.set_ylabel('Accuracy')
ax.set_title('Final Accuracy Comparison')
ax.set_ylim(0, 1)
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
for bar, val in zip(bars, accs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{val:.1%}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('m3c_v3_student_results.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Save results
results = {
    'phase': 'M3c_triple_prime_v2',
    'experiment': 'Curriculum-Optimized HDC Semantic Transfer',
    'node': 'student',
    'teacher_model': knowledge_packet.get('teacher_model'),
    'student_model': 'ImprovedHDCClassifier (ResidualBlocks)',
    'student_params': student.num_params,
    'hdc_dim': HDC_DIM,
    'curation_method': 'kmeans',
    'curriculum_strategy': 'sharp (easy→hard)',
    'constant_lr': LEARNING_RATE,
    'lr_decay': False,
    'num_easy': len(easy_examples),
    'num_hard': len(hard_examples),
    'teacher_accuracy_before': float(teacher_acc_before),
    'teacher_accuracy_after': float(teacher_acc_after),
    'student_accuracy_before': float(acc_before),
    'student_accuracy_after': float(acc_after),
    'transfer_efficiency_pct': float(transfer_efficiency),
    'baseline_efficiency_pct': 59.2,
    'improvement_over_baseline': float(transfer_efficiency - 59.2),
    'verdict': verdict,
    'insights_applied': [
        'M2.5c: HDC K-means curation',
        'M2.5e: Sharp curriculum (easy→hard)',
        'M2.5e: Constant LR (no decay)',
        'M2.5e: Distance from centroid = difficulty'
    ],
    'timestamp': datetime.now().isoformat()
}

with open('m3c_v3_student_results.json', 'w') as f:
    json.dump(results, f, indent=2)

ref = db.reference('sep_m3c_hdc_v3/student_results')
ref.set(results)

print("\n✅ Results saved!")
print(json.dumps(results, indent=2))

In [ ]:
from google.colab import files
files.download('m3c_v3_student_results.json')
files.download('m3c_v3_student_results.png')